Agent version example

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

# Set environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
model = ChatOpenAI(model="gpt-3.5-turbo")
embedding = OpenAIEmbeddings()

In [3]:
from langchain_community.vectorstores import Chroma
import document_handler

# https://python.langchain.com/docs/integrations/vectorstores/chroma

chroma_collection_name = "LangChainCollection"

import chromadb
new_client = chromadb.EphemeralClient()

vectorstore_initialize = Chroma.from_documents(
    document_handler.processed_texts,
    embedding=embedding,
    collection_name=chroma_collection_name,
    client=new_client,
)

vectorstore = Chroma(
    client=new_client,
    collection_name=chroma_collection_name,
    embedding_function=embedding,
)
retriever = vectorstore.as_retriever()

In [4]:
docs = vectorstore_initialize.similarity_search("What is Chocolate?")
print(docs)
docs = vectorstore.similarity_search("What is Chocolate?")
print(docs)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white 

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white 

In [4]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor, tool
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [5]:
from langchain.memory import ConversationSummaryBufferMemory
conversational_memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=model,
    memory_key = "chat_history",
    return_messages=True
)

In [46]:
# Prompt

system_message_template = (
    "You are a helpful assistant who helps answer questions. Answer only the facts based on the context. "
    "Your goal is to provide accurate and relevant answers based on the facts in the provided context. "
    "Make sure to reference the above source documents appropriately and avoid making assumptions or adding personal opinions. "
    "Emphasize the use of facts from the provided source documents. "
    "Instruct the model to use source name for each fact used in the response. "
    "Avoid generating speculative or generalized information. "
    "Use square brackets to reference the source, e.g. [info1.txt]. "
    "Do not combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\n"
    "Here is how you should answer every question:\n"
        "-Look for relevant information in the above source documents to answer the question.\n"
        "-If the source document does not include the exact answer, please respond with relevant information from the data in the response along with citation. You must include a citation to each document referenced.\n"
        "-If you cannot find answer in below sources, respond with I am not sure. Do not provide personal opinions or assumptions and do not include citations.\n"
        "-If you use any information in the context, include the index(starts at 1) of the statement as citation in your answer\n"
    "At the end of your response:\n" 
    "1. Add key words from the paragraphs. \n"
    "2. Suggest a further question that can be answered by the paragraphs provided. \n"
    "3. Create a source list of source name, author name, and a link for each document you cited.\n"
    "{context}"

)

MEMORY_KEY = "chat_history"

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message_template),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("human", "{question}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [47]:
from langchain.tools.retriever import create_retriever_tool
# tools
retriever_tool = create_retriever_tool(
    retriever,
    "search_docs",
    "Searches and returns documents for information to answer the question",
)

@tool
def retrieve_documents_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)[:3]
    # return docs
    texts = [doc.page_content for doc in docs]
    texts_merged = '\n\n'.join(texts)
    print("Retrieving documents")
    print(texts_merged)
    return texts_merged


tools = [retriever_tool]

In [48]:
# Set llm chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# agent = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | final_prompt
#     | model
#     | StrOutputParser()
# )
from langchain.agents.conversational_chat.base import ConversationalChatAgent

agent = ConversationalChatAgent.from_llm_and_tools(llm=model, tools=tools, prompt=final_prompt)

In [9]:
chat_history = []

In [49]:
agent_executor = AgentExecutor(tools=tools, agent=agent, verbose=True, memory=conversational_memory)

In [15]:
query = "What is chocolate?"
print(agent_executor.invoke({"input":query}))



> Entering new AgentExecutor chain...
{
    "action": "search_docs",
    "action_input": "What is chocolate?"
}

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3



Observation: Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.

Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.

Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate contains a higher percentage of cocoa solids and less sugar, giving it a more intense and bitter flavor. Milk chocolate, on the othe

In [28]:
import pandas as pd

df = pd.read_csv('./test_data/Questions.csv', delimiter=',')
tuples = [tuple(x) for x in df.values]
dicts = df.to_dict('records')

print(dicts)

[{'Question': 'What is Chocolate?', 'Chat_history': "{'chat_history': []}", 'Answer': 'Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter [info1.txt].'}, {'Question': 'Is that a word?', 'Chat_history': "{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.'}}", 'Answer': 'Yes, Chocolate is a word.'}, {'Question': 'Write the following words and then continue: g\\thumb|350', 'Chat_history': "{'chat_history': []}", 'Answer': 'I am not sure.'}

In [126]:
questions = list(map(lambda x : x['Question'], dicts))
print(questions)
for dict in dicts:
        question = dict['Question']
        follow_up = dict['Follow up']
        answer = dict['Answer']
        if follow_up == "No":
                memory.clear()
        print(memory.load_memory_variables({}))
        llm_response = agent_executor.invoke({"question": question})
        print(llm_response)
        memory.save_context({"question":question}, {"output":llm_response})

memory.clear()

['What is Chocolate?', 'Is that a word?', 'Write the following words and then continue: g\\thumb|350', 'What is iPhone?', 'What is Task Decomposition?']


KeyError: 'Follow up'

LangSmith Evaluation

In [12]:
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'

import uuid

from langsmith import Client

uid = uuid.uuid4()
client = Client()

In [ ]:
# from langsmith.evaluation import EvaluationResult, RunEvaluator
# from langsmith.schemas import Example, Run

# from langchain.chains import LLMChain
# from langchain_openai import ChatOpenAI


# class SentimentEvaluator(RunEvaluator):
#     def __init__(self):
#         prompt = """Is the predominant sentiment in the following statement positive, negative, or neutral?
# ---------
# Statement: {input}
# ---------
# Respond in one word: positive, negative, or neutral.
# Sentiment:"""

#         llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
#         self.chain = LLMChain.from_string(llm=llm, template=prompt)

#     def evaluate_run(self, run: Run, example: Example) -> EvaluationResult:
#         input_str = str(list(run.inputs.values())[0])
#         prediction = self.chain.run(input_str)
#         # Strip the prompt
#         prediction = prediction.strip()
#         score = {"positive": 1, "negative": -1, "neutral": 0}.get(prediction)
#         return EvaluationResult(
#             key="sentiment",
#             value=prediction,
#             score=score,
#         )

# evaluator = SentimentEvaluator()
# for run in client.list_runs(
#     project_name="my-project",
#     execution_order=1, # Do not return child / nested runs
# ):
#         client.evaluate_run(run, evaluator)


In [ ]:
# for question in questions:
#     result = chain.invoke(question)
#     # print(result)

#     question=result.get("input", None)
#     actual_output=result.get("output", None)
#     retrieval_context = [result.get("context", None)]
#     input = final_prompt.format(question=result["input"], context=result["context"])

#     print(input)
#     print(actual_output)
#     print(retrieval_context)

#     test_case = LLMTestCase(
#         input=final_prompt.format(question=input, context=retrieval_context),
#         actual_output=actual_output,
#         retrieval_context = retrieval_context
#     )

#     # print(assert_test(test_case, metrics))
#     test_cases.append(test_case)

# print(evaluate(test_cases, metrics))

In [29]:
# print([e for e in questions])
# print(dicts[0])
# print([dict['Chat_history'] for dict in dicts[2:]])
# print(dict)
import ast
from langchain.schema import AIMessage, HumanMessage, SystemMessage

def convert_dicts(input):
    # print(input)
#     print({'Question': 'Is that a word?',
#  'Chat_history': "{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.'}}",
#  'Answer': 'Yes, Chocolate is a word.'})
    chat_history = eval(input)
    # print(ast.literal_eval(input))
    return chat_history

# for dict in dicts:
#     print(convert_dicts(dict['Chat_history']))


print([convert_dicts(dict['Chat_history']) for dict in dicts])


# for dict_item in dicts:
#     chat_history_str = dict_item['Chat_history']
#     chat_history = eval(chat_history_str)
#     print(chat_history)

{'chat_history': []}
{'Question': 'Is that a word?', 'Chat_history': "{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.'}}", 'Answer': 'Yes, Chocolate is a word.'}
{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.'}}
{'Question': 'Is that a word?', 'Chat_history': "{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and cel

In [44]:
dataset_name = f"Simple agent langsmith evaluation dataset - {uid}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="An example agent evals dataset",
)
import ast

client.create_examples(
    inputs=[{"input": dict['Question'], "chat_history": convert_dicts(dict['Chat_history'])} for dict in dicts],
    # inputs=[{"question": dict['Question'], "chat_history": (dict['Chat_history'])} for dict in dicts],
    outputs=[{"output": dict['Answer']} for dict in dicts], # Outputs are optional, but recommended.
    dataset_id=dataset.id,
)

{'chat_history': []}
{'Question': 'Is that a word?', 'Chat_history': "{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.'}}", 'Answer': 'Yes, Chocolate is a word.'}
{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.'}}
{'Question': 'Is that a word?', 'Chat_history': "{'chat_history': {'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and cel

In [51]:
from langchain.smith import RunEvalConfig
from langchain.evaluation import load_evaluator, EvaluatorType

eval_config = RunEvalConfig(
    # We will use the chain-of-thought Q&A correctness evaluator
    evaluators=["cot_qa"],
)

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        # RunEvalConfig.Criteria({"creativity": "Is this submission creative, imaginative, or novel?"}),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness"),
        RunEvalConfig.Criteria("relevance"),
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria("coherence"),
        RunEvalConfig.Criteria("maliciousness"),
        RunEvalConfig.Criteria("helpfulness"),
        RunEvalConfig.Criteria("controversiality"),
        RunEvalConfig.Criteria("misogyny"),
        RunEvalConfig.Criteria("criminality"),
        RunEvalConfig.Criteria("insensitivity"),
        RunEvalConfig.Criteria("depth"),
        RunEvalConfig.Criteria("creativity"),
        RunEvalConfig.Criteria("detail"),
        # RunEvalConfig.Criteria(EvaluatorType.COT_QA)
    ],
    input_key="question"
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import ast

def construct_chain(input):
    # Add a step to convert the data from the dataset to a form the chain can consume
    
    new_memory = ConversationSummaryBufferMemory(
        k=4,
        max_token_limit=64,
        llm=model,
        memory_key = "chat_history",
        return_messages=True
    )
    example2 = input["chat_history"]
    print(example2["chat_history"])
    # example2 = convert_dicts(example)
    # print(example2['chat_history'])
    # print(example2["history"][0].content)
    # new_memory.save_context(input["chat_history"]["history"][0], input["chat_history"]["history"][1])
    # new_memory.save_context(example2["history"][0], example2["history"][1])
    if example2['chat_history'] == []:
        ""
    else:
    # new_memory.save_context(input["chat_history"]["chat_history"])
        print(example2["chat_history"]["input"])
        new_memory.save_context({"question":example2["chat_history"]["input"]}, {"answer":example2["chat_history"]["output"]})
    return AgentExecutor(tools=tools, agent=agent, verbose=True, memory=new_memory, handle_parsing_errors=True)


results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=construct_chain,
    evaluation=eval_config,
    project_name="simple_agent_testing-58",
    verbose = True,
)

#KEEP THIS URL https://smith.langchain.com/public/cfa11397-dafa-4936-9548-a984b33f1658/d
# https://smith.langchain.com/public/17ab53ee-6971-41dd-b8ed-a9b41390aed0/d

View the evaluation results for project 'simple_agent_testing-58' at:
https://smith.langchain.com/o/99caf4db-f19e-57e9-bc93-12af0e3dd026/datasets/349767c1-9faa-4f4f-8a4d-f9cfbc2aed9b/compare?selectedSessions=d5e50c8e-f6dc-461a-85c8-c626914e401f

View all tests for Dataset Simple agent langsmith evaluation dataset - 9ebe3b8c-8725-4933-b4d9-726eb30a08d8 at:
https://smith.langchain.com/o/99caf4db-f19e-57e9-bc93-12af0e3dd026/datasets/349767c1-9faa-4f4f-8a4d-f9cfbc2aed9b
[>                                                 ] 0/5{'input': 'What is chocolate?', 'output': 'Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.'}[]

What is chocolate?
[]
[]
[]


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new Agent

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


{
    "action": "search_docs",
    "action_input": "Task Decomposition"
}{
    "action": "search_docs",
    "action_input": "What is iPhone?"
}Could not parse LLM output: I apologize for the confusion. "Chocolate" is indeed a word. It refers to a delicious treat made from cocoa beans. If you have any more questions or need further assistance, feel free to ask!

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3



Observation: Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.

Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.

Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate contains a higher percentage of cocoa solids and less sugar, giving it a more intense and bitter flavor. Milk chocolate, on the othe

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3



Observation: Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.

Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.

Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate contains a higher percentage of cocoa solids and less sugar, giving it a more intense and bitter flavor. Milk chocolate, on the othe

,feedback.COT Contextual Accuracy,error,execution_time,run_id
count,0,0,5.000000,5
unique,0,0,NaN,5
top,NaN,NaN,NaN,21bdafbe-88f5-4f15-b2a1-8551fcaa27b8
freq,NaN,NaN,NaN,1
mean,NaN,NaN,8.835749,NaN
std,NaN,NaN,2.048408,NaN
min,NaN,NaN,6.295124,NaN
25%,NaN,NaN,7.051277,NaN
50%,NaN,NaN,9.556043,NaN
75%,NaN,NaN,10.384521,NaN


In [119]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
example = {'chat_history': {'history': [HumanMessage(content='What is Chocolate?'), AIMessage(content='Chocolate is a sweet food product that is usually brown in color and is made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is further processed to extract cocoa solids and cocoa butter [info1.txt].')]}}
example2 = example["chat_history"]["history"][0].content

example3 = {'chat_history': "{'history': [HumanMessage(content='What is Chocolate?'), AIMessage(content='Chocolate is a sweet food product that is usually brown in color and is made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is further processed to extract cocoa solids and cocoa butter [info1.txt].')]}"}
# example3 = {'chat_history': "[HumanMessage(content='What is Chocolate?'), AIMessage(content='Chocolate is a sweet food product that is usually brown in color and is made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is further processed to extract cocoa solids and cocoa butter [info1.txt].')]"}
import ast
example4 = "{'history': [HumanMessage(content='What is Chocolate?'), AIMessage(content='Chocolate is a sweet food product that is usually brown in color and is made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is further processed to extract cocoa solids and cocoa butter [info1.txt].')]}"
print(example4)
example5 = '"' + example4 + '"'
print(example5)
# print(f"{example4}".format({"example4": example4}))
converted_list = ast.literal_eval(example5)
print(converted_list)
print(example3["chat_history"])
# print(example4[0].content)
# converted_list = ast.literal_eval(example3["chat_history"])
print(converted_list["history"][0].content)

{'history': [HumanMessage(content='What is Chocolate?'), AIMessage(content='Chocolate is a sweet food product that is usually brown in color and is made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is further processed to extract cocoa solids and cocoa butter [info1.txt].')]}
"{'history': [HumanMessage(content='What is Chocolate?'), AIMessage(content='Chocolate is a sweet food product that is usually brown in color and is made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is further processed to extract cocoa solids and cocoa butter [info1.txt].')]}"
{'history': [HumanMessage(content='What is Chocolate?'), AIMessage(content='Chocolat

TypeError: string indices must be integers, not 'str'

In [102]:
import ast

string_representation = "{'history': ['random', 'random2']}"
converted_dict = ast.literal_eval(string_representation)

print(converted_dict)

{'history': ['random', 'random2']}


In [189]:
import ast
from langchain.schema import AIMessage, HumanMessage, SystemMessage

dicts = [
    {'Question': 'What is Chocolate?', 'Chat_history': "{'chat_history': []}", 'Answer': 'Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter [info1.txt].'},
    {'Question': 'Is that a word?', 'Chat_history': "{'chat_history': [HumanMessage(content='What is chocolate?'), AIMessage(content='Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.')]}", 'Answer': 'Yes, Chocolate is a word.'},
    {'Question': 'Write the following words and then continue: g\\thumb|350', 'Chat_history': "{'chat_history': []}", 'Answer': 'I am not sure.'},
    {'Question': 'What is iPhone?', 'Chat_history': "{'chat_history': []}", 'Answer': 'I am not sure.'},
    {'Question': 'What is Task Decomposition?', 'Chat_history': "{'chat_history': []}", 'Answer': 'I am not sure.'}
]

# Print evaluated 'Chat_history' values using ast.literal_eval()
for dict_item in dicts:
    chat_history_str = dict_item['Chat_history']
    chat_history = eval(chat_history_str)
    print(chat_history)

{'chat_history': []}
{'chat_history': [HumanMessage(content='What is chocolate?'), AIMessage(content='Chocolate is a sweet, usually brown, food product made from cocoa beans. It is enjoyed worldwide and is often associated with indulgence and celebration. There are various types of chocolate, including dark chocolate, milk chocolate, and white chocolate.')]}
{'chat_history': []}
{'chat_history': []}
{'chat_history': []}
